<center> <h1>Workshop: IV</h1> </center> 
<center> <h2>Application: IV estimation of demand function</h2> </center> 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from statsmodels.api import add_constant

In [2]:
from linearmodels.iv import IV2SLS

### 0. Setup
This is an example of demand estimation from Stock and Watson’s textbook. We want to estimate the demand for cigarettes using a dataset containing sales, prices and a number of other variables measured in different U.S. states. 
You can download the dataset from Canvas.
If you are using R, the dataset is contained in the AER package so you don’t need to download it separately.


**Description** A data frame containing 48 observations on 7 variables for 2 periods.

**state:**
Factor indicating state.

**year:**
Factor indicating year.

**cpi:**
Consumer price index.

**population:**
State population.

**packs:**
Number of packs per capita.

**income:**
State personal income (total, nominal).

**tax:**
Average state, federal and average local excise taxes for fiscal year.

**price:**
Average price during fiscal year, including sales tax.

**taxs:**
Average excise taxes for fiscal year, including sales tax.




In [3]:
cigs_all = pd.read_csv('cigarette.csv')
cigs_all.head()

,state,year,cpi,population,packs,income,tax,price,taxs
0,AL,1985,1.076,3973000.0,116.486282,46014968,32.500004,102.181671,33.348335
1,AR,1985,1.076,2327000.0,128.534592,26210736,37.000000,101.474998,37.000000
2,AZ,1985,1.076,3184000.0,104.522614,43956936,31.000000,108.578751,36.170418
3,CA,1985,1.076,26444000.0,100.363037,447102816,26.000000,107.837341,32.104000
4,CO,1985,1.076,3209000.0,112.963539,49466672,31.000000,94.266663,31.000000


In [4]:
# Following S&W, we use just 1995 data 
cigs = cigs_all[cigs_all.year==1995].copy()

### Create new variables

# deflate prices and income by CPI to get real values

cigs["rprice"] = cigs.price / cigs.cpi
cigs["rincome"] = (cigs.income/cigs.population)/cigs.cpi 

# log values are used in the regressions (note that you could create these on the fly within the regression commands)

cigs["lprice"]= np.log(cigs.rprice)
cigs["lquant"] = np.log(cigs.packs)
cigs["lincome"] = np.log(cigs.rincome)

# tdiff = the real tax on cigarettes arising just from general sales tax, which we will use as an instrument 
cigs["tdiff"]= (cigs.taxs - cigs.tax)/cigs.cpi


In [5]:
cigs.head()

,state,year,cpi,population,packs,income,tax,price,taxs,rprice,rincome,lprice,lquant,lincome,tdiff
48,AL,1995,1.524,4262731.0,101.085434,83903280,40.500004,158.371338,41.904671,103.918206,12.915347,4.643604,4.615966,2.558416,0.921697
49,AR,1995,1.524,2480121.0,111.042969,45995496,55.500000,175.542511,63.859169,115.185380,12.169073,4.746543,4.709917,2.498898,5.485019
50,AZ,1995,1.524,4306908.0,71.954170,88870496,65.333328,198.607498,74.790825,130.319887,13.539638,4.869992,4.276029,2.605622,6.205707
51,CA,1995,1.524,31493524.0,56.859306,771470144,61.000000,210.504669,74.771332,138.126430,16.073591,4.928169,4.040580,2.777178,9.036307
52,CO,1995,1.524,3738061.0,82.582924,92946544,44.000000,167.350006,44.000000,109.809720,16.315557,4.698749,4.413803,2.792119,0.000000


### 1. Naive Regression

**Q:** First run a regression of (log) packs of cigarettes on (log) price and comment on the estimate of the price elasticity you obtained.

In [6]:
naive_reg = smf.ols(formula = 'lquant ~ lprice', data = cigs).fit()
print(naive_reg.summary())

                            OLS Regression Results                            
Dep. Variable:                 lquant   R-squared:                       0.406
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     31.41
Date:                Thu, 21 Mar 2024   Prob (F-statistic):           1.13e-06
Time:                        01:47:24   Log-Likelihood:                 12.724
No. Observations:                  48   AIC:                            -21.45
Df Residuals:                      46   BIC:                            -17.71
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.3389      1.035      9.986      0.0

**A:** The estimated price elasticity is -1.21 (a 1% increase in price results in a 1.21% decrease in packs of cigarettes demanded). This is a biased estimate due to the fact that prices and quantity are determined in equilibrium.

### 2. First- and second-stage regressions for IV estimation

**Q:** Explain why sales tax (tdiff) could be a valid instrument.

**A:** 

- Relevance: high sales taxes increase the after-tax sales price, so these variables will be correlated. 
- Exogeneity: We need the sales tax to be uncorrelated with the error in the demand equation. Note that general sales tax rates vary from state to state mainly due to different state choices in terms of how the finance themselves. States choose different mixes of sales, income, property and other taxes as a source of revenue. These choices are driven by political choices rather than by the demand of cigarretes. (We will revisit this later when we think harder about the exogeneity assumption!)


**Q:** Using the sales tax (tdiff) as an instrument, start by running the first-stage regression for IV estimation. Comment on the results. Also generate the predicted values that you need for the second stage regression.

In [170]:
stage1 = smf.ols(formula = "lprice ~ tdiff", data = cigs).fit()
print(stage1.summary())

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.471
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     40.96
Date:                Wed, 16 Feb 2022   Prob (F-statistic):           7.27e-08
Time:                        11:57:09   Log-Likelihood:                 46.435
No. Observations:                  48   AIC:                            -88.87
Df Residuals:                      46   BIC:                            -85.13
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6165      0.029    158.601      0.0

In [171]:
cigs["stage1_prediction"] = stage1.predict()

**A:** The first-stage regression shows that the instrument is strong (F-statistic = 40.96 > 10). Note that in this case the F-statistic equals the t-statistic squared.

**Q:** Now run the second-stage regression for IV estimation. Compare your estimate of the demand elasticity to the one you obtained from the naive regression in part 1.

In [172]:
stage2 = smf.ols(formula = "lquant ~ stage1_prediction", data = cigs).fit()
print(stage2.summary())

                            OLS Regression Results                            
Dep. Variable:                 lquant   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     8.277
Date:                Wed, 16 Feb 2022   Prob (F-statistic):            0.00607
Time:                        11:57:10   Log-Likelihood:                 4.2040
No. Observations:                  48   AIC:                            -4.408
Df Residuals:                      46   BIC:                           -0.6656
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             9.7199      1.80

**A:** The elasticity estimate becomes -1.08: a 1% increase in price results in a 1.08% decrease in packs of cigarettes demanded.

### 3. TSLS estimation

#### 3.1 Simple TSLS
**Q:** Now run the TSLS estimation using the IV2SLS command from "linearmodels" and compare with the results in part 2.

In [173]:
from linearmodels.iv import IV2SLS
cigs = add_constant(cigs, has_constant="add")

In [174]:
iv = IV2SLS(cigs["lquant"], cigs[["const"]],  cigs["lprice"], cigs["tdiff"]).fit()
print(iv)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 lquant   R-squared:                      0.4011
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3881
No. Observations:                  48   F-statistic:                    12.046
Date:                Wed, Feb 16 2022   P-value (F-stat)                0.0005
Time:                        11:57:12   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          9.7199     1.4961     6.4966     0.0000      6.7875      12.652
lprice        -1.0836     0.3122    -3.4708     0.00

**A:** We obtain the same estimate of elasticity. Note that the standard error from IV2SLS is different from the one you computed manually (the one from the IV2SLS command is the right one, as it takes into account the fact that in the second stage regression the X-variable depends on estimated parameters from the first-stage regression - i.e., it is a “generated regressor”).

#### 3.2 More Sources of Bias

**Q:**  Let's revisit the exogeneity assumption. Do you think income ("lincome") is an omitted variable that could be affecting the validity of our instrument? Why? 

**A:** Notes that stats with higher incomes may depend less on sales taxes and more on income taxes. That is, sales taxes can be correlated with state income. At the same time, the demand for cigarretes may depend on income. 

**Q:** Now include log income (the variable lincome you generated in the beginning) in the regression as a control variable. Comment on the results.

In [175]:
iv2 = IV2SLS(cigs["lquant"], cigs[["const","lincome"]],  cigs["lprice"], cigs["tdiff"]).fit()
print(iv2)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 lquant   R-squared:                      0.4189
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3931
No. Observations:                  48   F-statistic:                    17.474
Date:                Wed, Feb 16 2022   P-value (F-stat)                0.0002
Time:                        11:57:12   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          9.4307     1.2194     7.7338     0.0000      7.0407      11.821
lincome        0.2145     0.3018     0.7107     0.47

In [176]:
print(iv2.first_stage)

    First Stage Estimation Results    
                                lprice
--------------------------------------
R-squared                       0.6389
Partial R-squared               0.5009
Shea's R-squared                0.5009
Partial F-statistic             47.713
P-value (Partial F-stat)     4.935e-12
Partial F-stat Distn           chi2(1)
========================== ===========
const                           3.5908
                              (21.471)
lincome                         0.3893
                              (6.1484)
tdiff                           0.0274
                              (6.9074)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [177]:
from linearmodels.iv import compare

In [178]:
print(compare({"IV": iv, "IV controling for income": iv2}))

                      Model Comparison                     
                                IV IV controling for income
-----------------------------------------------------------
Dep. Variable               lquant                   lquant
Estimator                  IV-2SLS                  IV-2SLS
No. Observations                48                       48
Cov. Est.                   robust                   robust
R-squared                   0.4011                   0.4189
Adj. R-squared              0.3881                   0.3931
F-statistic                 12.046                   17.474
P-value (F-stat)            0.0005                   0.0002
==================     ===========              ===========
const                       9.7199                   9.4307
                          (6.4966)                 (7.7338)
lprice                     -1.0836                  -1.1434
                         (-3.4708)                (-3.1718)
lincome                                 

**A:** The estimate of the elasticity goes down. This is expected as state income was an omitted variable, which is positively correlated with quantity of cigarettes (as we see from its coefficient in the regression), and also plausibly positively correlated with cigarette price. This means we were overestimating the elasticity in the regression without income.

#### 3.3 More Instruments

**Q:** Re-run the model in 3.2, also considering another instrument in addition to tdiff, obtained as the real tax on cigarette: tax/cpi. Comment on the estimates of both the price elasticity and the income elasticity of cigarette demand.

In [179]:
cigs["rtax"]= cigs.tax/cigs.cpi

In [180]:
iv3 = IV2SLS(cigs["lquant"], cigs[["const","lincome"]],  cigs["lprice"], cigs[["tdiff","rtax"]]).fit()
print(iv3)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 lquant   R-squared:                      0.4294
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4041
No. Observations:                  48   F-statistic:                    34.506
Date:                Wed, Feb 16 2022   P-value (F-stat)                0.0000
Time:                        11:57:14   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          9.8950     0.9288     10.654     0.0000      8.0746      11.715
lincome        0.2804     0.2458     1.1407     0.25

In [181]:
print(compare({"IV": iv, "IV controling for income": iv2,"IV income + additional instrument": iv3}))

                                       Model Comparison                                      
                                IV IV controling for income IV income + additional instrument
---------------------------------------------------------------------------------------------
Dep. Variable               lquant                   lquant                            lquant
Estimator                  IV-2SLS                  IV-2SLS                           IV-2SLS
No. Observations                48                       48                                48
Cov. Est.                   robust                   robust                            robust
R-squared                   0.4011                   0.4189                            0.4294
Adj. R-squared              0.3881                   0.3931                            0.4041
F-statistic                 12.046                   17.474                            34.506
P-value (F-stat)            0.0005                   0.0002 

**A:** The elasticity estimate goes down even further and it is significant. Provided the instruments are valid, we have a more precise estimate of the demand function (we have more “as if random” variation with two valid instruments than with one instrument). We can test the validity of the two instruments below. We conclude that there is high elasticity of cigarette demand to price, but not to income, whose elasticity estimate is not significant.

#### 3.4 Instrument Validity
**Q:** Assess the validity of the instruments. Using:

- iv3.first_stage
- iv3.sargan


(In R adding the diagnostics=TRUE to the summary command in the regression in part 3.3.)

In [182]:
print(iv3.first_stage)

    First Stage Estimation Results   
                               lprice
-------------------------------------
R-squared                      0.9403
Partial R-squared              0.9175
Shea's R-squared               0.9175
Partial F-statistic            457.48
P-value (Partial F-stat)       0.0000
Partial F-stat Distn          chi2(2)
========================== ==========
const                          4.1030
                             (48.489)
lincome                        0.1083
                             (2.8539)
tdiff                          0.0109
                             (5.3235)
rtax                           0.0094
                             (11.230)
-------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [183]:
#testing for Exogeneity
print(iv3.sargan)

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 0.3326
P-value: 0.5641
Distributed: chi2(1)


**A:** We reject weak instruments (good) and fail to reject exogeneity (good).

**Bonus Question:** You are thinking about using the "distance of the state from cigarrete manufacturing plants" as an instrument. Do you think this would be a weak or strong instrument? Why?

**A:** More distance implies increased shipping costs which means increased cigarrete prices. However, shipping costs are likely to be a small portion of the price of cigarrete (cigarretes are lightweight). In turn, shipping costs (a proxy for distance) would explain a very small fraction of the price variation and thus could be a weak instrument. 